# Plotting points on Google Maps

In [1]:
import json

import pandas as pd

from bokeh.io import output_file, output_notebook, show
from bokeh.models import GMapPlot, GMapOptions, ColumnDataSource, Circle, PanTool, WheelZoomTool, HoverTool
from bokeh.plotting import gmap

output_notebook()

Loading BokehJS ...

## I. Load data from NYC Open Data's API

In [2]:
## Most of the datasets on NYC's Open Data Portal can be accessed through the Socrata API, but you have to sign up for
## an access token to request more than 1,000 rows at a time, so I'm just grabbing the csv
## I chose this dataset pretty randomly -- my only requirement was that it contained lat/lon data

df = pd.read_csv("https://data.cityofnewyork.us/api/views/s4kf-3yrf/rows.csv?accessType=DOWNLOAD")
df

,CB Link ID,Borough,Community Board,Council District,Latitude,Longitude,Project Status,Smallest PPT,Street Address,Postcode,Link Site ID,Link Smoke Tested and Activated (A),Link Installation (A),Neighborhood Tabulation Area (NTA),Building Identification Number (BIN),Borough-Block-Lot (BBL),Census Tract (CT),Location
0,LINK-019315,Bronx,205,14,40.850173,-73.910282,Live,146097.0,1 EAST 177 STREET,10453,bx-05-146097,07/13/2018,06/20/2018,Mount Hope,2008247,2028530001,2023301,"(40.850173, -73.91028173)"
1,LINK-008842,Manhattan,105,4,40.747060,-73.985250,Live,121611.0,1 EAST 32 STREET,10016,mn-05-121611,12/09/2016,10/01/2016,Midtown-Midtown South,1017006,1008620001,1007400,"(40.74705974, -73.98525002)"
2,LINK-008843,Manhattan,105,4,40.747680,-73.984799,Live,121613.0,1 EAST 33 STREET,10016,mn-05-121613,03/23/2017,01/29/2017,Midtown-Midtown South,1017076,1008630006,1007400,"(40.74767999, -73.98479875)"
3,LINK-000186,Manhattan,105,2,40.734801,-73.989085,Live,134620.0,1 IRVING PLACE,10003,mn-05-134620,07/19/2017,06/11/2017,Gramercy,1083247,1008707501,1005000,"(40.7348011, -73.9890846)"
4,LINK-010750,Manhattan,102,3,40.730216,-74.006646,Live,123600.0,1 ST. LUKE'S PLACE,10014,mn-02-123600,01/26/2017,11/13/2016,West Village,1009773,1005830001,1006700,"(40.73021595, -74.0066461)"
5,LINK-021138,Brooklyn,301,34,40.709572,-73.950706,Live,146047.0,1 TEN EYCK STREET,11206,bk-01-146047,03/29/2018,12/19/2017,North Williamsburg,3069651,3027910001,3051300,"(40.70957163, -73.95070637)"
6,LINK-000214,Manhattan,105,3,40.738129,-73.992700,Live,123023.0,1 WEST 17 STREET,10011,mn-05-123023,02/20/2018,02/11/2017,Hudson Yards-Chelsea-Flatiron-Union Square,1015419,1008190037,1005400,"(40.73812904, -73.99269997)"
7,LINK-010173,Manhattan,105,4,40.762503,-73.974937,Live,122853.0,1 WEST 56 STREET,10019,mn-05-122853,03/28/2017,10/29/2016,Midtown-Midtown South,1035051,1012720034,1011201,"(40.76250264, -73.97493691)"
8,LINK-011588,Bronx,205,14,40.853824,-73.907505,Live,119613.0,1 WEST BURNSIDE AVENUE,10453,bx-05-119613,01/12/2017,11/23/2016,University Heights-Morris Heights,2014153,2031920075,2025100,"(40.85382365, -73.90750467)"
9,LINK-012181,Manhattan,105,3,40.736965,-73.993793,Live,137100.0,10 WEST 15 STREET,10011,mn-05-137100,05/09/2018,09/23/2017,Hudson Yards-Chelsea-Flatiron-Union Square,1078686,1008160031,1005400,"(40.73696532, -73.99379346)"


## II. Style Google Maps

In [3]:
## This part is completely unnecessary if you're okay with using the regular Google Maps, but I think it's kind of 
## ugly so I'm using a styler from snazzymaps.com

styles_json = [
    {
        "featureType": "water",
        "elementType": "geometry",
        "stylers": [
            {
                "color": "#e9e9e9"
            },
            {
                "lightness": 17
            }
        ]
    },
    {
        "featureType": "landscape",
        "elementType": "geometry",
        "stylers": [
            {
                "color": "#f5f5f5"
            },
            {
                "lightness": 20
            }
        ]
    },
    {
        "featureType": "road.highway",
        "elementType": "geometry.fill",
        "stylers": [
            {
                "color": "#ffffff"
            },
            {
                "lightness": 17
            }
        ]
    },
    {
        "featureType": "road.highway",
        "elementType": "geometry.stroke",
        "stylers": [
            {
                "color": "#ffffff"
            },
            {
                "lightness": 29
            },
            {
                "weight": 0.2
            }
        ]
    },
    {
        "featureType": "road.arterial",
        "elementType": "geometry",
        "stylers": [
            {
                "color": "#ffffff"
            },
            {
                "lightness": 18
            }
        ]
    },
    {
        "featureType": "road.local",
        "elementType": "geometry",
        "stylers": [
            {
                "color": "#ffffff"
            },
            {
                "lightness": 16
            }
        ]
    },
    {
        "featureType": "poi",
        "elementType": "geometry",
        "stylers": [
            {
                "color": "#f5f5f5"
            },
            {
                "lightness": 21
            }
        ]
    },
    {
        "featureType": "poi.park",
        "elementType": "geometry",
        "stylers": [
            {
                "color": "#dedede"
            },
            {
                "lightness": 21
            }
        ]
    },
    {
        "elementType": "labels.text.stroke",
        "stylers": [
            {
                "visibility": "on"
            },
            {
                "color": "#ffffff"
            },
            {
                "lightness": 16
            }
        ]
    },
    {
        "elementType": "labels.text.fill",
        "stylers": [
            {
                "saturation": 36
            },
            {
                "color": "#333333"
            },
            {
                "lightness": 40
            }
        ]
    },
    {
        "elementType": "labels.icon",
        "stylers": [
            {
                "visibility": "off"
            }
        ]
    },
    {
        "featureType": "transit",
        "elementType": "geometry",
        "stylers": [
            {
                "color": "#f2f2f2"
            },
            {
                "lightness": 19
            }
        ]
    },
    {
        "featureType": "administrative",
        "elementType": "geometry.fill",
        "stylers": [
            {
                "color": "#fefefe"
            },
            {
                "lightness": 20
            }
        ]
    },
    {
        "featureType": "administrative",
        "elementType": "geometry.stroke",
        "stylers": [
            {
                "color": "#fefefe"
            },
            {
                "lightness": 17
            },
            {
                "weight": 1.2
            }
        ]
    }
]

In [4]:
json_string = json.dumps(styles_json)

## III. Make map!

In [5]:
## Set the center of the map to somewhere in NYC & apply the map style 

map_options = GMapOptions(lat=40.7589, 
                          lng=-73.9, 
                          map_type="roadmap", 
                          zoom=12, 
                          styles=json_string) 

## Using Google Maps in Bokeh requires a Google API key from 
## https://developers.google.com/maps/documentation/javascript/get-api-key
API_KEY = "your_api_key"

plot = gmap(API_KEY, 
            map_options=map_options, 
            title="Locations of LinkNYC Kiosks", 
            plot_width=1000, 
            plot_height=800)

plot.title.text_font = "nunito"
plot.title.align = "center"
plot.title.text_font_size="40px"

source = ColumnDataSource(data=dict(
                                    lat=df['Latitude'].tolist(),
                                    lon=df['Longitude'].tolist()
                                    )
                        )

plot.circle(x="lon", 
            y="lat", 
            size=3, 
            line_color="#ff9900", 
            fill_color="#ff9900", 
            fill_alpha=0.8, 
            source=source)

show(plot)
